In [1]:
import pandas as pd
import altair as alt
import geopandas as gpd

In [2]:
alt.data_transformers.enable('default', max_rows=None) 

DataTransformerRegistry.enable('default')

In [3]:
df = pd.read_csv('./gapminder.csv')
df.head()

,country,continent,year,gdpPerCap,lifeExp,pop
0,Afghanistan,Asia,2009,1454.663015,60.484,28394813
1,Afghanistan,Asia,2010,1637.377987,61.028,29185507
2,Afghanistan,Asia,2011,1626.764793,61.553,30117413
3,Afghanistan,Asia,2012,1806.763930,62.054,31161376
4,Afghanistan,Asia,2013,1874.765634,62.525,32269589


In [7]:
slider = alt.binding_range(min=2009, max=2017, step=1, name='year:')
slider_selection = alt.selection_single(name="year_selector", fields=['year'],bind=slider, init={'year': 2016})

country_selection = alt.selection_single(fields=['continent'])
map_selection = alt.selection_single(empty='all', fields=['continent'], on='mouseover')

gdf = gpd.read_file('world.geojson')
gdf.crs = {'init' :'epsg:27700'}
gdf = gdf.to_crs({'init': 'epsg:4326'})

data_geojson = alt.InlineData(values=gdf.to_json(), format=alt.DataFormat(property='features',type='json')) 

colours_condition = alt.condition(map_selection | country_selection,
                    alt.Color('continent:N', legend=None),
                    alt.value('lightgray'))

back_left = alt.Chart(df).mark_circle().encode(
    x = alt.X('gdpPerCap'),
    y = alt.Y('lifeExp'),
    color =alt.value('lightgray'),
    size=alt.Size('pop', scale=alt.Scale(domain=[80000, 170000000], range=[10,300]))
).add_selection(slider_selection).transform_filter(slider_selection).properties(width = 600, height = 600)

left = alt.Chart(df).mark_circle().encode(
    x = alt.X('gdpPerCap'),
    y = alt.Y('lifeExp'),
    color = colours_condition,
    size=alt.Size('pop', scale=alt.Scale(domain=[80000, 170000000], range=[10,300])),
).transform_filter(slider_selection).add_selection(map_selection).properties(width = 600, height = 600)

right = alt.Chart(data_geojson).mark_geoshape(
).transform_calculate(continent="datum.properties.region_un").encode(
    color=alt.condition(country_selection,
                    alt.Color('continent:N', legend=None),
                    alt.value('lightgray'))
).add_selection(country_selection)

alt.hconcat(back_left + left, right)

alt.HConcatChart(...)